In [27]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [28]:
import netCDF4 as nc
import gsw
import pickle as pk
import numpy as np

In [32]:
# load temperature and salinity from SODA netcdf file

SODA_file = "/d1/enrique/bdry/NWA_bdry_SODA3.3.1_y2012.nc"
Sds = nc.Dataset(SODA_file)
#Sds.variables.keys()

In [33]:
# pull out salinity and temperature 

temp_south = Sds.variables['temp_south'][:]
temp_east = Sds.variables['temp_east'][:]

salt_south = Sds.variables['salt_south'][:]
salt_east = Sds.variables['salt_east'][:]

#print(temp_south.shape)
#print(temp_east.shape)
#print(salt_south.shape)
#print(salt_east.shape)

# east is 362, south is 722

In [34]:
# I previously calculated the depth at each point using grid file

with open('/d1/mkelly/NWA/romsdepth.p', 'rb') as f:
    depth = pk.load(f)

# index boundary depths
depth_east = depth[:, 721, :]
depth_south = depth[1, :, :]

# transpose depth to match other variable dims
depth_south=np.transpose(depth_south)
depth_east=np.transpose(depth_east)

#print(depth_south.shape)

# pickle depth
with open('BC_depth.p', 'wb') as f:
    pk.dump([depth_south, depth_east], f)

In [35]:
# get latitude and longitude coordinates from NWA grid file
grd_file = "/d1/mkelly/NWA/NWA_grd.nc"
grdds = nc.Dataset(grd_file)

lat = grdds.variables['lat_rho'][:]
lon = grdds.variables['lon_rho'][:]

#print(lon.shape)

# index BC coords
lat_south = lat[0, :]
lon_south = lon[0, :]

lat_east = lat[:, 721]
lon_east = lon[:, 721]

#print(lon_south.shape)
#print(lat_east.shape)

# repeat lat and lon at all 40 depth levels
lon_south = np.tile(lon_south, (40, 1))
lat_south = np.tile(lat_south, (40, 1))

lon_east = np.tile(lon_east, (40, 1))
lat_east = np.tile(lat_east, (40, 1))

# pickle coords
with open('BC_coords.p', 'wb') as f:
    pk.dump([lon_south, lon_east, lat_south, lat_east], f)

#print(lon_south.shape)
#print(lat_south.shape)

In [37]:
# pull out one time step

salt_south1 = salt_south[0, :, :]
salt_east1 = salt_east[0, :, :]

temp_south1 = temp_south[0, :, :]
temp_east1 = temp_east[0, :, :]

#print(salt_south1.shape)
#print(temp_south1.shape)

# will need to loop this to perform on all time steps

# need absolute salinity
#### SA_south = gsw.SA_from_SP(SP, p, lon, lat)
#### inputs are SP, p, lon, lat
#### should all have the same dims
#### every point has salinity, pressure, lon, and lat

In [39]:
# calculate absolute salinity

SA_south = gsw.SA_from_SP(salt_south1, depth_south, lon_south, lat_south)
SA_east = gsw.SA_from_SP(salt_east1, depth_east, lon_east, lat_east)

#print(SA_south.shape)

In [40]:
# convert potential to conservative temperature

CT_south = gsw.CT_from_pt(SA_south, temp_south1)
CT_east = gsw.CT_from_pt(SA_east, temp_east1)

In [41]:
# convert conservative to in-situ temperature

ist_south = gsw.t_from_CT(SA_south, CT_south, depth_south)
ist_east = gsw.t_from_CT(SA_east, CT_east, depth_east)

In [42]:
# MLR physical inputs are practical salinity and in situ temp
# pickle physical inputs
with open('SODA_inputs.p', 'wb') as f:
    pk.dump([salt_south1, salt_east1, ist_south, ist_east], f)